In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('./sales.csv')
df

,Age,Income,Gender,MaritialStatus,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes
5,>35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,<21,Medium,Male,Single,No
8,<21,Low,Female,Married,Yes
9,>35,Medium,Female,Single,Yes


In [14]:
train_df = df.iloc[:-1].copy()
test_df = df.iloc[-1:].copy()

In [15]:
class Node:
  def __init__(self, feature, values):
    self.feature = feature
    self.values = values
    self.yes = None
    self.no = None

  def __str__(self):
    return f'Feature: {self.feature}, Values: {self.values}'

class DecisionTree:
  def __gini(self, yes_count, no_count):
    yes_total = yes_count[0] + yes_count[1]
    no_total = no_count[0] + no_count[1]
    try:
      gini_yes = 1 - (yes_count[0] / yes_total) ** 2 - (yes_count[1] / yes_total) ** 2
    except: gini_yes = 0
    try:
      gini_no = 1 - (no_count[0] / no_total) ** 2 - (no_count[1] / no_total) ** 2
    except: gini_no = 0
    return (yes_total * gini_yes  + no_total * gini_no) / (yes_total + no_total)
  
  def __get_impurity(self, X, y, values):
    yes_count = [0, 0]
    no_count = [0, 0]
    for i in range(len(X)):
      if X[i] in values:
        if y[i]: yes_count[1] += 1
        else: yes_count[0] += 1
      else:
        if y[i]: no_count[1] += 1
        else: no_count[0] += 1
    return self.__gini(yes_count, no_count)

  def __parse(self, x, n):
    b = bin(x)[2:]
    b = (n-len(b)) * '0' + b
    val = list(b)
    return [i for i in range(len(val)) if val[i] == '1']

  def __get_feature_impurity(self, X, feature, y):
    values = self.unique_feature_values[feature]
    n = 2 ** len(values) - 1
    best_impurity = 100
    for i in range(1, n):
      idx = self.__parse(i, len(values))
      val_subset = values[idx].copy()
      impurity = self.__get_impurity(X, y, val_subset)
      # print("impurity = ", impurity, "feature = ", feature, " subset = ", val_subset)
      if impurity <= best_impurity:
        best_impurity = impurity
        best_values = val_subset    
    return best_values, best_impurity
  
  def __select_best_feature(self, X, y):
    best_impurity = 100
    for feature in X.columns:
      values, impurity = self.__get_feature_impurity(list(X[feature]), feature, y)
      # print(values, impurity)
      if impurity <= best_impurity:
        best_impurity = impurity
        best_feature = feature
        best_values = values
    return best_feature, best_values, best_impurity

  def __filter_data(self, X, y, feature, values, flag):
    X_filtered = X[X[feature].isin(values) == flag].copy()
    idx = list(X_filtered.index)
    X_filtered = X_filtered.reset_index().drop([feature, 'index'], axis = 1)
    y_filtered = y[idx].copy()
    return X_filtered, y_filtered
  
  def __build_tree(self, X, y, parent_impurity = 100):
    if len(X.columns) == 0: return None
    best_feature, best_values, impurity = self.__select_best_feature(X, y)
    print(best_feature, best_values, impurity)
    if impurity >= parent_impurity: return None
    node = Node(best_feature, best_values)
    
    X_yes, y_yes = self.__filter_data(X, y, best_feature, best_values, True)
    X_no, y_no = self.__filter_data(X, y, best_feature, best_values, False)
    
    node.yes = self.__build_tree(X_yes, y_yes, impurity)
    node.no = self.__build_tree(X_no, y_no, impurity)

    if node.yes is None: node.yes = True
    if node.no is None: node.no = False
    return node

  def __set_feature_values(self, X):
    self.unique_feature_values = {}
    for feature in X.columns:
      self.unique_feature_values[feature] = np.unique(list(X[feature]))

  def fit(self, X, y):
    self.__set_feature_values(X)
    self.tree = self.__build_tree(X, y)

  def __make_prediction(self, x, node):
    if type(node) == bool: return node
    
    value = x[node.feature]
    if value in node.values: node = node.yes
    else: node = node.no
    
    return self.__make_prediction(x, node)

  def predict(self, X):
    preds = []
    for i in range(len(X)):
      preds.append(self.__make_prediction(X.iloc[i], self.tree))
    return np.array(preds)
  
  def preorderprint(self, node, level):
    if type(node) == bool:
      print(node)
      return
    print(node.feature, node.values)
    print("no of ", node.feature, " level = ", level)
    self.preorderprint(node.no, level + 1)
    print("yes of ", node.feature, " level = ", level)
    self.preorderprint(node.yes, level + 1)

In [16]:
X_train, y_train = train_df.drop('Buys', axis = 1), np.array(train_df['Buys']) == 'Yes'
X_test = test_df.drop('Buys', axis = 1)
clf = DecisionTree()
clf.fit(X_train, y_train)
print(clf.predict(X_test))
print(f'Root: {clf.tree.feature}, Values: {clf.tree.values}')
clf.preorderprint(clf.tree, 0)

Age ['21-35' '>35'] 0.31923076923076926
MaritialStatus ['Married'] 0.16666666666666666
Gender ['Female'] 0.3333333333333333
Gender ['Female'] 0.0
Income ['High' 'Low'] 0.0
Income ['High' 'Low'] 0.0
Gender ['Female'] 0.0
MaritialStatus ['Married'] 0.0
MaritialStatus ['Married'] 0.0
[ True]
Root: Age, Values: ['21-35' '>35']
Age ['21-35' '>35']
no of  Age  level =  0
Gender ['Female']
no of  Gender  level =  1
False
yes of  Gender  level =  1
True
yes of  Age  level =  0
MaritialStatus ['Married']
no of  MaritialStatus  level =  1
Gender ['Female']
no of  Gender  level =  2
False
yes of  Gender  level =  2
True
yes of  MaritialStatus  level =  1
True
